In [1]:
import sklearn.metrics
# note Grakel does not seem to support Python >=3.10, Python 3.9 works fine
# you are free to remove imports that are not useful for you
from grakel.datasets import fetch_dataset
from grakel.kernels import WeisfeilerLehman, VertexHistogram
from sklearn.model_selection import train_test_split,cross_val_score
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.decomposition import KernelPCA # to check your own implementation
from sklearn.manifold import TSNE
import numpy as np
import scipy
import matplotlib.pyplot as plt
import math

In [2]:
# Some datasets, more datasets here https://ls11-www.cs.tu-dortmund.de/staff/morris/graphkerneldatasets

"""
    The MUTAG dataset consists of 188 chemical compounds divided into two 
    classes according to their mutagenic effect on a bacterium. 

    The chemical data was obtained form http://cdb.ics.uci.edu and converted 
    to graphs, where vertices represent atoms and edges represent chemical 
    bonds. Explicit hydrogen atoms have been removed and vertices are labeled
    by atom type and edges by bond type (single, double, triple or aromatic).
    Chemical data was processed using the Chemistry Development Kit (v1.4).
"""

"""
    ENZYMES is a dataset of protein tertiary structures obtained from (Borgwardt et al., 2005) 
    consisting of 600 enzymes from the BRENDA enzyme database (Schomburg et al., 2004). 
    In this case the task is to correctly assign each enzyme to one of the 6 EC top-level 
    classes. 
"""

"""
    NCI1 and NCI109 represent two balanced subsets of datasets of chemical compounds screened 
    for activity against non-small cell lung cancer and ovarian cancer cell lines respectively
    (Wale and Karypis (2006) and http://pubchem.ncbi.nlm.nih.gov).
"""

#fetch the data for the three datasets
datasetMutag = fetch_dataset("MUTAG", verbose=False) # just replace by the name of the datasets you want "ENZYMES", "NCI1"
GMutag = datasetMutag.data
yMutag = datasetMutag.target

datasetEnzimes = fetch_dataset("ENZYMES", verbose=False) # just replace by the name of the datasets you want "ENZYMES", "NCI1"
GEnzimes = datasetEnzimes.data
yEnzimes = datasetEnzimes.target

datasetNCI1 = fetch_dataset("NCI1", verbose=False) # just replace by the name of the datasets you want "ENZYMES", "NCI1"
GNCI1 = datasetNCI1.data
yNCI1 = datasetNCI1.target

## 4.1 Computing the kernels

In [3]:
H = 10
def pairwiseKernelComputation(graph, H):

    wl = WeisfeilerLehman(n_iter=H, base_graph_kernel=VertexHistogram,normalize=False)
    #pairwiseComputation
    wlTrain = wl.fit_transform(graph)

    return wlTrain



In [4]:
#4.1.2 et 4.1.4 MUTAG
pairwiseMutag = pairwiseKernelComputation(GMutag, H)
WlrankMutag = np.linalg.matrix_rank(pairwiseMutag)

#4.1.2 et 4.1.4 ENZIMES
pairwiseEnzimes = pairwiseKernelComputation(GEnzimes, H)
WlrankEnzimes = np.linalg.matrix_rank(pairwiseEnzimes)

#4.1.2 et 4.1.4 NCI1
pairwiseNCI1 = pairwiseKernelComputation(GNCI1, H)
WlrankNCI1 = np.linalg.matrix_rank(pairwiseNCI1)

## 4.2 Vizualisation

### 4.2.1 Kernel centralisation

In [5]:
#K = K − 1N K − K1N + 1N K1N
def kernelCentralisation(kernelMatrix):
    inverseMatrix = np.full((len(kernelMatrix[0]), len(kernelMatrix[0])), 1/len(kernelMatrix[0]))
    centredKernelMatrix = (kernelMatrix - inverseMatrix) @ (kernelMatrix - kernelMatrix@inverseMatrix) + (inverseMatrix@kernelMatrix@inverseMatrix)
    return centredKernelMatrix
centredKernelMutag = kernelCentralisation(pairwiseMutag)

### 4.2.2 KernelPCA implementation

In [6]:
vK = scipy.linalg.eig(centredKernelMutag) #compute eigenvalues,eigenvectors


### 4.2.3 KernelPCA visualization

### 4.2.4 Distance

### 4.2.5 tSNE

### 4.2.6 Comparison

## 4.3 Classification

### 4.3.1 A simple baseline

### 4.3.2 Support Vector Machine (SVM)

In [7]:
C = 1e2
H = 3
def assessAccuracy(graph, y, H, C):

    wl = WeisfeilerLehman(n_iter=H, base_graph_kernel=VertexHistogram,normalize=False)
    X_train, X_test, y_train, y_test = train_test_split(graph, y, test_size=0.20, random_state=1, shuffle=True)

    wlTrain = wl.fit_transform(X_train)
    wlTest = wl.transform(X_test)
    clf = SVC(kernel="precomputed", C=C)
    clf.fit(wlTrain, y_train)
    ypred = clf.predict(wlTest)
    acc = accuracy_score(y_test, ypred)
    print("Accuracy:", str(round(acc*100, 2)) + "%")

    return acc


In [8]:
accMutag = assessAccuracy(GMutag, yMutag, H, C)
accEnzimes = assessAccuracy(GEnzimes, yEnzimes, H, C)
accNCI1 = assessAccuracy(GNCI1, yNCI1, H, C)

Accuracy: 92.11%
Accuracy: 55.0%
Accuracy: 82.0%


### 4.3.3 Hyperparameters selection

In [9]:
C = [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1e0, 1e1, 1e2, 1e3, 1e4]
H = [i for i in range(1, 11, 1)]
def bestParams(graph, y, H, C):
    bestAcc = 0
    bestP = (0, 0)
    for i in range(len(H)):
        for j in range(len(C)):
            wl = WeisfeilerLehman(n_iter=H[i], base_graph_kernel=VertexHistogram,normalize=False)
            wlTrain = wl.fit_transform(graph)

            acc = cross_val_score(SVC(kernel="precomputed", C=C[j]), wlTrain, y, cv=10)
            #print(np.mean(acc))
            #print(bestAcc)
            if np.mean(acc) > bestAcc:
                bestAcc = np.mean(acc)
                bestP = (i, j)

    return bestAcc, bestP

In [ ]:

bestParamMutag = bestParams(GMutag, yMutag, H, C)
print(f"Best param Mutag: {bestParamMutag}")
bestParamEnzimes = bestParams(GEnzimes, yEnzimes, H, C)
print(f"Best param Enzimes: {bestParamEnzimes}")
bestParamNCI1 = bestParams(GNCI1, yNCI1, H, C)
print(f"Best param NCI1: {bestParamNCI1}")


Best param Mutag: (0.8932748538011696, (0, 5))


### 4.3.4 Oberservations